In [ ]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point

In [38]:
df1 = pd.read_csv("D:/DEPI_DATA/data/df_large/df_large.csv",index_col=False)
df2 = pd.read_csv("D:/DEPI_DATA/data/df_small/df_small.csv",index_col=False)

In [39]:
df1_unigue = df1[['LAT', 'LON']].drop_duplicates()
df2_unigue = df2[['LAT', 'LON']].drop_duplicates()


In [40]:
df1_unigue


,LAT,LON
0,22.0,25.000
5,22.0,35.000
10,22.0,25.625
15,22.0,35.625
20,22.0,26.250
...,...,...
3315,32.0,31.875
3325,32.0,32.500
3335,32.0,33.125
3345,32.0,33.750


In [41]:
df2_unigue

,LAT,LON
0,22.5,24.5
5,22.5,34.5
10,22.5,25.5
15,22.5,35.5
20,22.5,26.5
...,...,...
1055,31.5,30.5
1065,31.5,31.5
1075,31.5,32.5
1085,31.5,33.5


In [42]:
df2

,Unnamed: 0,LAT,LON,YEAR,DOY,AIRMASS,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,CLOUD_AMT,CLOUD_AMT_DAY,CLOUD_AMT_NIGHT,CLRSKY_DAYS,MIDDAY_INSOL,PSH,PW
0,0,22.5,24.5,2019,1,3.87,15.71,0.85,0.02,1.14,37.06,47.33,28.38,0.0,63.47,0.38,0.70
1,0,22.5,24.5,2020,1,3.87,16.53,0.90,0.02,1.16,22.80,9.57,34.00,0.0,66.77,0.40,1.05
2,0,22.5,24.5,2021,1,3.86,17.93,0.96,0.02,1.23,14.62,15.97,13.48,0.0,68.26,0.43,0.66
3,0,22.5,24.5,2022,1,3.86,16.45,0.90,0.02,1.14,10.43,13.11,8.17,0.0,61.22,0.39,1.08
4,0,22.5,24.5,2023,1,3.87,15.34,0.77,0.02,1.11,4.05,5.34,2.95,1.0,66.04,0.37,0.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222767,40255,31.5,30.5,2020,366,3.81,12.20,0.64,0.01,0.54,27.96,22.33,31.97,0.0,48.22,0.29,1.42
222768,40256,31.5,31.5,2020,366,3.82,11.29,0.60,0.01,0.51,39.59,36.89,41.52,0.0,44.76,0.27,1.35
222769,40257,31.5,32.5,2020,366,3.85,12.00,0.61,0.01,0.51,17.16,11.07,21.51,0.0,48.89,0.29,1.29
222770,40258,31.5,33.5,2020,366,3.92,11.91,0.61,0.01,0.50,17.98,11.71,22.46,0.0,48.74,0.29,1.29


In [43]:


# Load Egypt's boundary
egypt_boundary = gpd.read_file("https://raw.githubusercontent.com/datasets/geo-boundaries-world-110m/master/countries.geojson")

# Filter Egypt's boundary
egypt_polygon = egypt_boundary[egypt_boundary['admin'] == 'Egypt'].geometry.iloc[0]

# Extract the coordinates of the current Egypt polygon
coords = list(egypt_polygon.exterior.coords)


# Add the new points to the coordinate list
updated_coords = coords 

# Create a new polygon with the updated coordinates
updated_egypt_polygon = Polygon(updated_coords)


In [44]:

# Create a base map centered on Egypt
egypt_map = folium.Map(location=[26.5, 30], zoom_start=6)
points_within_polygon = []
# Plot all points inside Egypt as rectangles with labels
for _, row in df1_unigue.iterrows():
    lat, lon = row['LAT'], row['LON']
    
    # Check if the point is within Egypt's boundary
    point = gpd.points_from_xy([lon], [lat])[0]
    
    if updated_egypt_polygon.contains(point):  # Check if point is inside the polygon
        points_within_polygon.append((lat, lon)) 
        # Add rectangle
        folium.CircleMarker(
    location=[lat, lon],
    radius=4,  # Adjust this to change the marker size
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.8,
    tooltip=f"Lat: {lat}, Lon: {lon}",
    popup=f"Coordinates: Lat {lat}, Lon {lon}"
).add_to(egypt_map)

# Add the specific points to the map
specific_points = [(33.75, 27), (33.75, 27.5),(33.75,31.0),(34.375,26.0),(26.875,31.5),(28.75,31.0),(35.0,25.0),(35.625,23.5),(34.375,28.0),(34.375,28.5),(25.0,31.5),(32.5,29.5),(31.875,31.5)]

for lon, lat in specific_points:
    folium.CircleMarker(
    location=[lat, lon],
    radius=4,  # Adjust this to change the marker size
    color='red',
    fill=True,
    fill_color='blue',
    fill_opacity=0.8,
    tooltip=f"Lat: {lat}, Lon: {lon}",
    popup=f"Coordinates: Lat {lat}, Lon {lon}"
).add_to(egypt_map)
# Save the map to an HTML file
egypt_map.save("egypt_grid_map.html")

# Display the map if using Jupyter Notebook
egypt_map


In [45]:
total_points_in_Egypt=points_within_polygon+specific_points


# Convert to DataFrame
df3 = pd.DataFrame(total_points_in_Egypt, columns=['Latitude', 'Longitude'])

# Display the DataFrame
display(df3)
df3.to_csv("df3.csv",index=False)


,Latitude,Longitude
0,22.500,35.000
1,22.500,35.625
2,22.500,36.250
3,23.000,35.000
4,23.000,35.625
...,...,...
284,34.375,28.000
285,34.375,28.500
286,25.000,31.500
287,32.500,29.500


In [46]:

# Create a base map centered on Egypt
egypt_map_small = folium.Map(location=[26.5, 30], zoom_start=6)
points_within_polygon_small = []
# Plot all points inside Egypt as rectangles with labels
for _, row in df2_unigue.iterrows():
    lat, lon = row['LAT'], row['LON']
    
    # Check if the point is within Egypt's boundary
    point_small = gpd.points_from_xy([lon], [lat])[0]
    
    if updated_egypt_polygon.contains(point_small):  # Check if point is inside the polygon
        points_within_polygon_small.append((lat, lon)) 
        # Add rectangle
        folium.CircleMarker(
    location=[lat, lon],
    radius=4,  # Adjust this to change the marker size
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.8,
    tooltip=f"Lat: {lat}, Lon: {lon}",
    popup=f"Coordinates: Lat {lat}, Lon {lon}"
).add_to(egypt_map_small)

# Add the specific points to the map
specific_points_small = [(27.5,31.5),(32.5,29.5),(31.5,31.5),(32.5,31.5),(33.5,27.5),(35.5,24.5)]

for lon, lat in specific_points_small:
    folium.CircleMarker(
    location=[lat, lon],
    radius=4,  # Adjust this to change the marker size
    color='red',
    fill=True,
    fill_color='blue',
    fill_opacity=0.8,
    tooltip=f"Lat: {lat}, Lon: {lon}",
    popup=f"Coordinates: Lat {lat}, Lon {lon}"
).add_to(egypt_map_small)
# Save the map to an HTML file
egypt_map_small.save("egypt_grid_map_small.html")

# Display the map if using Jupyter Notebook
egypt_map_small


In [47]:
total_points_in_Egypt_small=points_within_polygon_small+specific_points_small


# Convert to DataFrame
df5 = pd.DataFrame(total_points_in_Egypt_small, columns=['Latitude', 'Longitude'])

# Display the DataFrame
print(df5)

    Latitude  Longitude
0       22.5       34.5
1       22.5       25.5
2       22.5       35.5
3       22.5       26.5
4       22.5       27.5
..       ...        ...
91      32.5       29.5
92      31.5       31.5
93      32.5       31.5
94      33.5       27.5
95      35.5       24.5

[96 rows x 2 columns]


In [48]:
# Filter rows from df1 based on matching Latitude and Longitude in df3
df_filtered_small = df2[df2.apply(lambda row: (row['LAT'], row['LON']) in zip(df5['Latitude'], df5['Longitude']), axis=1)]

# Display the filtered DataFrame
print("Filtered DataFrame (rows matching Latitude and Longitude pairs in df3):")
print(df_filtered_small)


Filtered DataFrame (rows matching Latitude and Longitude pairs in df3):
        Unnamed: 0   LAT   LON  YEAR  DOY  AIRMASS  ALLSKY_SFC_SW_DWN  \
5                0  22.5  34.5  2019    1     4.11              13.55   
6                0  22.5  34.5  2020    1     4.11              14.45   
7                0  22.5  34.5  2021    1     4.10              17.54   
8                0  22.5  34.5  2022    1     4.11              17.67   
9                0  22.5  34.5  2023    1     4.11              13.29   
...            ...   ...   ...   ...  ...      ...                ...   
222760       40248  30.5  34.5  2020  366     3.81              13.17   
222762       40250  31.5  25.5  2020  366     6.22              13.14   
222763       40251  31.5  26.5  2020  366     4.09              11.37   
222767       40255  31.5  30.5  2020  366     3.81              12.20   
222768       40256  31.5  31.5  2020  366     3.82              11.29   

        ALLSKY_SFC_UVA  ALLSKY_SFC_UVB  ALLSKY_SFC_

In [49]:
# Sort DataFrame by 'YEAR' and 'DOY' in ascending order

df_sorted_small = df_filtered_small.sort_values(by=['YEAR', 'DOY',"LAT","LON"], ascending=[True, True,True,True])

# Display the sorted DataFrame
print(df_sorted_small)

# Optionally save to CSV
df_sorted_small.to_csv('sorted_data_small.csv', index=False)


        Unnamed: 0   LAT   LON  YEAR  DOY  AIRMASS  ALLSKY_SFC_SW_DWN  \
10               1  22.5  25.5  2019    1     3.93              14.94   
20               2  22.5  26.5  2019    1     4.02              13.70   
30               3  22.5  27.5  2019    1     4.14              12.29   
40               4  22.5  28.5  2019    1     4.29              11.38   
50               5  22.5  29.5  2019    1     4.46              11.21   
...            ...   ...   ...   ...  ...      ...                ...   
222606       40138  30.5  34.5  2023  365     3.83              12.66   
222616       40140  31.5  25.5  2023  365     6.24              11.73   
222621       40141  31.5  26.5  2023  365     4.09              12.07   
222641       40145  31.5  30.5  2023  365     3.83              11.22   
222646       40146  31.5  31.5  2023  365     3.83              10.68   

        ALLSKY_SFC_UVA  ALLSKY_SFC_UVB  ALLSKY_SFC_UV_INDEX  CLOUD_AMT  \
10                0.81            0.02           

In [50]:
df3.sort_values(by="Latitude",ascending=True)
df3.to_csv("df3.csv",index=False)

In [51]:
# Filter rows from df1 based on matching Latitude and Longitude in df3
df_filtered_large = df1[df1.apply(lambda row: (row['LAT'], row['LON']) in zip(df3['Latitude'], df3['Longitude']), axis=1)]

# Display the filtered DataFrame
print("Filtered DataFrame (rows matching Latitude and Longitude pairs in df3):")
print(df_filtered_large)


Filtered DataFrame (rows matching Latitude and Longitude pairs in df3):
        Unnamed: 0   LAT     LON  YEAR  DOY  EVLAND  GWETPROF  GWETROOT  \
45               4  22.5  35.000  2019    1    0.03      0.33      0.33   
46               4  22.5  35.000  2020    1    0.15      0.33      0.33   
47               4  22.5  35.000  2021    1    0.13      0.33      0.33   
48               4  22.5  35.000  2022    1    0.28      0.34      0.33   
49               4  22.5  35.000  2023    1    0.43      0.34      0.34   
...            ...   ...     ...   ...  ...     ...       ...       ...   
679237      122941  31.0  33.125  2020  366    0.13      0.18      0.17   
679241      122945  31.5  25.625  2020  366    0.94      0.57      0.59   
679242      122946  31.5  26.250  2020  366    1.12      0.59      0.62   
679249      122953  31.5  30.625  2020  366    0.57      0.50      0.49   
679250      122954  31.5  31.250  2020  366    0.36      0.61      0.60   

        GWETTOP  PRECSNO  .

In [52]:
# Sort DataFrame by 'YEAR' and 'DOY' in ascending order

df_sorted = df_filtered_large.sort_values(by=['YEAR', 'DOY',"LAT","LON"], ascending=[True, True,True,True])

# Display the sorted DataFrame
print(df_sorted)

# Optionally save to CSV
df_sorted.to_csv('sorted_data_large.csv', index=False)


        Unnamed: 0   LAT     LON  YEAR  DOY  EVLAND  GWETPROF  GWETROOT  \
170             17  22.5  25.625  2019    1    0.00      0.37      0.36   
180             18  22.5  26.250  2019    1    0.00      0.27      0.27   
190             19  22.5  26.875  2019    1    0.00      0.20      0.20   
200             20  22.5  27.500  2019    1    0.00      0.15      0.15   
210             21  22.5  28.125  2019    1    0.00      0.20      0.20   
...            ...   ...     ...   ...  ...     ...       ...       ...   
678765      122605  31.0  33.125  2023  365    0.13      0.14      0.14   
678785      122609  31.5  25.625  2023  365    0.14      0.35      0.35   
678790      122610  31.5  26.250  2023  365    0.09      0.34      0.33   
678825      122617  31.5  30.625  2023  365    0.92      0.51      0.51   
678830      122618  31.5  31.250  2023  365    0.98      0.64      0.64   

        GWETTOP  PRECSNO  ...  TS_MIN   WD2M  WD50M  WS2M  WS2M_MAX  WS2M_MIN  \
170        0.09   